# CRS Math Profs Database

By:

Mariano, Isaiah

Montealto, Meluisa

Regalario, Jeremiah

# Import Dependencies

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import re
import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import matplotlib.pyplot as plt
import random

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Webscrape Data from CRS

## Links

In [2]:
first_sem = "https://crs.upd.edu.ph/schedule/1{}1/Math"
second_sem = "https://crs.upd.edu.ph/schedule/1{}2/Math"
midyr = "https://crs.upd.edu.ph/schedule/1{}4/Math"

## Enter Year Range

In [3]:
first_year = 2018
last_year = 2024

In [4]:
links = []
for i in range(first_year, last_year + 1):
  exec(f"AY{i}{i + 1}A = first_sem.format(i)")
  exec(f"AY{i}{i + 1}B = second_sem.format(i)")
  exec(f"AY{i}{i + 1}M = midyr.format(i)")
  exec(f"links.append(AY{i}{i + 1}A)")
  exec(f"links.append(AY{i}{i + 1}B)")
  exec(f"links.append(AY{i}{i + 1}M)")

In [5]:
AY20242025B

'https://crs.upd.edu.ph/schedule/120242/Math'

In [6]:
links

['https://crs.upd.edu.ph/schedule/120181/Math',
 'https://crs.upd.edu.ph/schedule/120182/Math',
 'https://crs.upd.edu.ph/schedule/120184/Math',
 'https://crs.upd.edu.ph/schedule/120191/Math',
 'https://crs.upd.edu.ph/schedule/120192/Math',
 'https://crs.upd.edu.ph/schedule/120194/Math',
 'https://crs.upd.edu.ph/schedule/120201/Math',
 'https://crs.upd.edu.ph/schedule/120202/Math',
 'https://crs.upd.edu.ph/schedule/120204/Math',
 'https://crs.upd.edu.ph/schedule/120211/Math',
 'https://crs.upd.edu.ph/schedule/120212/Math',
 'https://crs.upd.edu.ph/schedule/120214/Math',
 'https://crs.upd.edu.ph/schedule/120221/Math',
 'https://crs.upd.edu.ph/schedule/120222/Math',
 'https://crs.upd.edu.ph/schedule/120224/Math',
 'https://crs.upd.edu.ph/schedule/120231/Math',
 'https://crs.upd.edu.ph/schedule/120232/Math',
 'https://crs.upd.edu.ph/schedule/120234/Math',
 'https://crs.upd.edu.ph/schedule/120241/Math',
 'https://crs.upd.edu.ph/schedule/120242/Math',
 'https://crs.upd.edu.ph/schedule/120244

In [7]:
link = links[0]
identifier = link.split("/")[4]
identifier

'120181'

## Webscraping using BeautifulSoup

In [8]:
# Function to map semester numbers to names
def convert_semester(sem):
    mapping = {
        '1': '1st sem',
        '2': '2nd sem',
        '4': 'Midyear'
    }
    return mapping.get(sem, 'Unknown')

In [9]:
for link in links:

  page = requests.get(link)
  soup = BeautifulSoup(page.content, "html.parser")

  table = soup.find("table")
  headers = []
  for header in table.find_all('th'):
      headers.append(header.text.strip())
  rows = []
  for row in table.find_all('tr'):
      cells = row.find_all(['td', 'th'])
      cells = [cell.text.strip() for cell in cells]
      rows.append(cells)
  identifier = link.split("/")[4]
  exec("df{} = pd.DataFrame(rows, columns=headers)".format(identifier))
  exec("df{}['Year'] = identifier[1:5]".format(identifier)) #Start year
  exec("df{}['Semester'] = identifier[-1]".format(identifier)) #Semester
  exec("df{}['Semester'] = df{}['Semester'].apply(convert_semester)".format(identifier, identifier))
  exec("df{} = df{}.drop(df{}.index[0])".format(identifier, identifier, identifier))

In [10]:
df120231.head()

,Class Code,Class,Credits,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Enlisting Unit : BlockBlock Remarks,Available Slots /\n\t\tTotal Slots,Demand,Restrictions,Year,Semester
1,39233,Math 2 THR,3.0,"TTh 8:30-10AM lec MBAN 306\n\t\t\tCONSORTE, OD...",MATH,0 /\n\t\t\t\t51,0,"For: BS HE(2 slots), AIT(3 slots)",2023,1st sem
2,39235,Math 2 THU,3.0,"TTh 10-11:30AM lec MBAN 306\n\t\t\tAGUILAR, AD...",MATH,0 /\n\t\t\t\t52,0,,2023,1st sem
3,39430,Math 2 THU-1,3.0,"TTh 10-11:30AM lec MBAN 312\n\t\t\tLUY, JASMIN...",MATH,0 /\n\t\t\t\t30,0,,2023,1st sem
4,39234,Math 2 THW,3.0,"TTh 1-2:30PM lec MBAN 306\n\t\t\tABARA, MA. NE...",MATH,0 /\n\t\t\t\t56,0,,2023,1st sem
5,39242,Math 2 WFU,3.0,"WF 10-11:30AM lec MBAN 306\n\t\t\tGALVEZ, LUCK...",MATH,0 /\n\t\t\t\t51,0,"For: BA Film(1 slots), BA BMAS(1 slots), SLIS(...",2023,1st sem


In [11]:
len(df120231)

303

## Concatenate DataFrames

In [12]:
for link in links[1:]:
  identifier = link.split("/")[4]
  exec("df120181 = pd.concat([df120181, df{}])".format(identifier))
df = df120181.copy()

In [13]:
len(df)

4328

# Data Cleaning

In [14]:
df.head()

,Class Code,Class,Credits,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Enlisting Unit : BlockBlock Remarks,Available Slots /\n\t\tTotal Slots,Demand,Restrictions,Year,Semester
1,39236,Math 2 THR,3.0,"TTh 8:30-10AM lec TBA\n\t\t\tWALO, MA. LAILANI...",MATH,1 /\n\t\t\t\t64,0,,2018,1st sem
2,39398,Math 2 THU,3.0,"TTh 10-11:30AM lec TBA\n\t\t\tCEJO, ROBERT JAY...",MATH,0 /\n\t\t\t\t59,0,,2018,1st sem
3,39238,Math 2 THV,3.0,"TTh 11:30AM-1PM lec TBA\n\t\t\tWALO, MA. LAILA...",MATH,0 /\n\t\t\t\t63,0,,2018,1st sem
4,39237,Math 2 WFV,3.0,"WF 11:30AM-1PM lec TBA\n\t\t\tAGUILAR, ADJANI\...",MATH,0 /\n\t\t\t\t65,0,,2018,1st sem
5,39315,Math 10 THR,3.0,"TTh 8:30-10AM lec TBA\n\t\t\tARCEO, CARLENE PE...",MATH,0 /\n\t\t\t\t97,0,,2018,1st sem


## Selected Columns

In [15]:
df = df[['Class', 'Schedule\n\t\tInstructor(s)\n\t\tRemarks', 'Year', 'Semester']]
df.head()

,Class,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Year,Semester
1,Math 2 THR,"TTh 8:30-10AM lec TBA\n\t\t\tWALO, MA. LAILANI...",2018,1st sem
2,Math 2 THU,"TTh 10-11:30AM lec TBA\n\t\t\tCEJO, ROBERT JAY...",2018,1st sem
3,Math 2 THV,"TTh 11:30AM-1PM lec TBA\n\t\t\tWALO, MA. LAILA...",2018,1st sem
4,Math 2 WFV,"WF 11:30AM-1PM lec TBA\n\t\t\tAGUILAR, ADJANI\...",2018,1st sem
5,Math 10 THR,"TTh 8:30-10AM lec TBA\n\t\t\tARCEO, CARLENE PE...",2018,1st sem


## Math Class Number

In [16]:
df['Number'] = df['Class'].str.split().str[1]
df.head()

,Class,Schedule\n\t\tInstructor(s)\n\t\tRemarks,Year,Semester,Number
1,Math 2 THR,"TTh 8:30-10AM lec TBA\n\t\t\tWALO, MA. LAILANI...",2018,1st sem,2
2,Math 2 THU,"TTh 10-11:30AM lec TBA\n\t\t\tCEJO, ROBERT JAY...",2018,1st sem,2
3,Math 2 THV,"TTh 11:30AM-1PM lec TBA\n\t\t\tWALO, MA. LAILA...",2018,1st sem,2
4,Math 2 WFV,"WF 11:30AM-1PM lec TBA\n\t\t\tAGUILAR, ADJANI\...",2018,1st sem,2
5,Math 10 THR,"TTh 8:30-10AM lec TBA\n\t\t\tARCEO, CARLENE PE...",2018,1st sem,10


In [17]:
df['Number'].unique()

array(['2', '10', '11', '14', '17', '20', '21', '30', '40', '53', '54',
       '55', '100', '102', '109', '110.1', '110.2', '110.3', '117', '121',
       '122', '123.1', '123.2', '128', '147', '150.1', '162', '164',
       '171', '180.1', '197', '200', '201', '202.1', '203', '205', '208',
       '209.1', '210.1', '211', '214', '220.1', '221', '222', '228',
       '229', '236', '240', '242', '243', '246', '260', '262.1', '265',
       '271.1', '280', '290', '294', '296', '297', '300', '400', '22',
       '108', '140', '142', '146', '148', '150.2', '196', '202.2', '204',
       '209.2', '210.2', '218', '220.2', '235', '241', '247', '250',
       '261', '262.2', '266', '271.2', '281', '295', '23', '217', '288',
       '249', '216', '133', '158', '190', '227', '126', '166', '282',
       '131', '258', '180.2', '262', '268.2'], dtype=object)

## Remove / and None

In [18]:
df = df[df['Number'] != '/']
len(df)

4328

In [19]:
df = df[df['Number'].isna() == False]
len(df)

4328

## Create Necessary Columns


In [20]:
# Function to extract day, time, and professor
def extract_info(schedule):
    day_pattern = r'\b(M|TTh|WF|S|TWThF)\b'
    time_pattern = r'\b\d{1,2}(?::\d{2})?(?:AM|PM)?-\d{1,2}(?::\d{2})?(?:AM|PM)?\b'
    prof_pattern = r'(?<=\n\t\t\t).*$'
    room_pattern = r'\b(TBA|MBAN \d{3}|MB \d{3})\b'

    prof_match = re.search(prof_pattern, schedule, re.MULTILINE | re.DOTALL)
    prof = prof_match.group().strip() if prof_match else ''

    day = re.search(day_pattern, schedule)
    time = re.search(time_pattern, schedule)
    room = re.search(room_pattern, schedule)

    return (
        day.group() if day else '',
        time.group() if time else '',
        room.group() if room else 'TBA',
        prof
    )

# Apply the function to the DataFrame
df[['Day', 'Time',  'Room','Prof']] = df['Schedule\n\t\tInstructor(s)\n\t\tRemarks'].apply(lambda x: pd.Series(extract_info(x)))

In [21]:
df['Prof'] = df['Prof'].str.split('\n').str[0]

In [22]:
df = df.dropna(subset=['Prof'])  # Drop rows where 'Prof' is NaN
df = df.dropna(subset=['Prof'])  # Drop rows where 'Time' is NaN
df = df[~df['Prof'].str.upper().isin(['CONCEALED', 'TBA'])]

In [23]:
df = df[["Number", "Day", "Time","Room", "Prof",  "Year", "Semester"]]

In [24]:
df.head()

,Number,Day,Time,Room,Prof,Year,Semester
1,2,TTh,8:30-10AM,TBA,"WALO, MA. LAILANI",2018,1st sem
2,2,TTh,10-11:30AM,TBA,"CEJO, ROBERT JAY",2018,1st sem
3,2,TTh,11:30AM-1PM,TBA,"WALO, MA. LAILANI",2018,1st sem
4,2,WF,11:30AM-1PM,TBA,"AGUILAR, ADJANI",2018,1st sem
5,10,TTh,8:30-10AM,TBA,"ARCEO, CARLENE PERPETUA",2018,1st sem


## Convert to Military Time

In [25]:
def to_military_time(time, suffix):
    if "AM" not in time and "PM" not in time:
        if ":" in time:
            hour, minute = time.split(":")
            hour = int(hour)
            minute = int(minute)
            if suffix == "PM" and hour < 8:
                hour += 12
            return f"{hour:02}:{minute:02}"
        else:
            try:
              time = int(time)
              if suffix == "PM" and time < 8:
                time += 12
              return f"{time:02}:00"
            except:
              return f"e: {time} 1"
    elif time[-2:] == "AM":
        time = time[:-2]
        if ":" in time:
            hour, minute = time.split(":")
            hour = int(hour)
            minute = int(minute)
            return f"{hour:02}:{minute:02}"
        else:
            try:
              time = int(time)
              return f"{time:02d}:00"
            except:
              return f"e: {time} 2"
    elif time[-2:] == "PM":
        time = time[:-2]
        if ":" in time:
            hour, minute = time.split(":")
            hour = int(hour)
            if hour < 8:
              hour += 12
            minute = int(minute)
            return f"{hour:02}:{minute:02}"
        else:
            try:
              time = int(time)
              if time < 8:
                time += 12
              return f"{time:02}:00"
            except:
              return f"e: {time} 3"

def split_time(time):
    time = str(time)
    if "-" not in time:
        return "", ""
    start_time, end_time = time.split('-')
    start_time = str(start_time)
    end_time = str(end_time)
    suffix = time[-2:]
    return to_military_time(start_time, suffix), to_military_time(end_time, suffix)

In [26]:
df["Time"].unique()

array(['8:30-10AM', '10-11:30AM', '11:30AM-1PM', '1-2:30PM', '2:30-4PM',
       '7-8:30AM', '2:30-3:45PM', '7:15-8:15AM', '8:45-9:45AM',
       '1:15-2:15PM', '2:45-3:45PM', '10:15-11:15AM', '11:45AM-12:45PM',
       '8:30-9:45AM', '1-2:15PM', '10-11:15AM', '11:30AM-12:45PM',
       '4-5:30PM', '12-3PM', '8:30-11:30AM', '', '5:30-7PM', '10AM-1PM',
       '4-7PM', '9AM-12PM', '7-10AM', '3-4PM', '4-5PM', '1-4PM',
       '4:15-5:15PM', '3-6PM', '7-9AM', '9-11AM', '11AM-1PM',
       '9:35AM-12:10PM', '12:10-2:45PM', '2:45-5:20PM', '7-9:35AM',
       '10:15AM-1:30PM', '1-3PM', '7-8:15AM', '5-8PM', '2-5PM', '12-1PM',
       '1-2PM', '10-11AM', '2:30-5:30PM', '8-11AM', '11:30AM-2:30PM',
       '9-10AM', '11AM-12:15PM', '3-4:15PM', '9-10:15AM', '2:45-4:30PM',
       '7-8:45AM', '9:35-11:20AM', '12:10-1:55PM', '5-6:15PM', '6-9PM',
       '4-6PM', '7:30-9:30AM', '10:30AM-12:30PM', '6-7PM', '5-6PM',
       '3-5PM'], dtype=object)

In [27]:
df[["Start_time", "End_time"]] = df["Time"].apply(lambda x: pd.Series(split_time(x)))

## Remove Original Time column

In [28]:
df = df.drop(columns=["Time"])
df.head()

,Number,Day,Room,Prof,Year,Semester,Start_time,End_time
1,2,TTh,TBA,"WALO, MA. LAILANI",2018,1st sem,08:30,10:00
2,2,TTh,TBA,"CEJO, ROBERT JAY",2018,1st sem,10:00,11:30
3,2,TTh,TBA,"WALO, MA. LAILANI",2018,1st sem,11:30,13:00
4,2,WF,TBA,"AGUILAR, ADJANI",2018,1st sem,11:30,13:00
5,10,TTh,TBA,"ARCEO, CARLENE PERPETUA",2018,1st sem,08:30,10:00


# Save to CSV

In [29]:
df.to_csv("CRS Math Profs (2018-2024).csv")